In [1]:
import random
import numpy as np
import pandas as pd
import copy
import time
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

In [2]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
        # helper variables
        self.bias_nitem = sum(sizes[1:])
        self.weight_nitem = sum([self.weights[i].size for i in range(self.num_layers-2)])

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w,a)+b)
        return a

    def sigmoid(self, z):
        return 1.0/(1.0+np.exp(-z))

    def score(self, X, y):
        total_score=0
        for i in range(X.shape[0]):
            predicted = self.feedforward(X[i].reshape(-1,1))
            actual = y[i].reshape(-1,1)
            total_score += np.sum(np.power(predicted-actual,2))  # mean-squared error
        return total_score/X.shape[0]

    def accuracy(self, X, y):
        accuracy = 0
        for i in range(X.shape[0]):
            output = (self.feedforward(X[i].reshape(-1,1))).reshape(-1)
            condition = True
            for j in range(len(output)):
                output[j] = round(output[j])
            for j in range(len(output)):
                if(output[j]!=y[i][j]):
                    condition = False
                    break
            if condition:
                accuracy += 1
        return accuracy / X.shape[0] * 100

    def __str__(self):
        s = "\nBias:\n\n" + str(self.biases)
        s += "\nWeights:\n\n" + str(self.weights)
        s += "\n\n"
        return s

In [3]:
class NNGeneticAlgo:

    def __init__(self, n_pops, net_size, mutation_rate, crossover_rate, X, y, X_test, y_test):
        self.n_pops = n_pops
        self.net_size = net_size
        self.nets = [Network(self.net_size) for i in range(self.n_pops)]
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.X = X[:]
        self.y = y[:]
        self.X_test = X_test[:]
        self.y_test = y_test[:]
        self.accuracy_train = []
        self.accuracy_test = []
        self.best = Network(self.net_size)
            
    def get_random_point(self, type):
        nn = self.nets[0]
        layer_index, point_index = random.randint(0, nn.num_layers-2), 0
        if type == 'weight':
            row = random.randint(0,nn.weights[layer_index].shape[0]-1)
            col = random.randint(0,nn.weights[layer_index].shape[1]-1)
            point_index = (row, col)
        elif type == 'bias':
            point_index = random.randint(0,nn.biases[layer_index].size-1)
        return (layer_index, point_index)

    def get_all_scores(self,Xc,yc):
        return [net.score(Xc,yc) for net in self.nets]

    def get_all_accuracy(self,Xc,yc):
        return [net.accuracy(Xc,yc) for net in self.nets]

    def crossover(self):
        for i in range(self.n_pops):
            if random.uniform(0,1) < self.crossover_rate:
                father = random.randint(0,self.n_pops-1)
                mother = random.randint(0,self.n_pops-1)
                # make a copy of father 'genetic' weights & biases information
                nn_1 = copy.deepcopy(self.nets[father])
                nn_2 = copy.deepcopy(self.nets[mother])
                # cross-over bias
                k_1 = random.randint(int(0.5*self.nets[0].bias_nitem),self.nets[0].bias_nitem)
                for _ in range(k_1):
                    # get some random points
                    layer, point = self.get_random_point('bias')
                    # replace genetic (bias) with mother's value
                    nn_1.biases[layer][point] = self.nets[mother].biases[layer][point]
                    nn_2.biases[layer][point] = self.nets[father].biases[layer][point]

                # cross-over weight
                k_2 = random.randint(int(0.5*self.nets[0].weight_nitem),self.nets[0].weight_nitem)
                for _ in range(k_2):
                    # get some random points
                    layer, point = self.get_random_point('weight')
                    # replace genetic (weight) with mother's value
                    nn_1.weights[layer][point] = self.nets[mother].weights[layer][point]
                    nn_2.weights[layer][point] = self.nets[father].weights[layer][point]
                self.nets.append(copy.deepcopy(nn_1))
                self.nets.append(copy.deepcopy(nn_2))
        
    def mutation(self):
        for i in range(self.n_pops):
            if random.uniform(0,1) < self.mutation_rate:
                origin = random.randint(0,self.n_pops-1)
                nn = copy.deepcopy(self.nets[origin])

                # mutate bias
                k_1 = random.randint(int(0.5*self.nets[0].bias_nitem),self.nets[0].bias_nitem)
                for _ in range(k_1):
                    # get some random points
                    layer, point = self.get_random_point('bias')
                    # add some random value between -0.5 and 0.5
                    nn.biases[layer][point] += random.uniform(-0.5, 0.5)

                # mutate weight
                k_2 = random.randint(int(0.5*self.nets[0].weight_nitem),self.nets[0].weight_nitem)
                for _ in range(k_2):
                    # get some random points
                    layer, point = self.get_random_point('weight')
                    # add some random value between -0.5 and 0.5
                    nn.weights[layer][point[0], point[1]] += random.uniform(-0.5, 0.5)
                self.nets.append(copy.deepcopy(nn))
        
    def selection(self,Xc,yc):
        nets_new=[]
        for i in range(self.n_pops):
            k_1 = random.randint(0,len(self.nets)-1)
            k_2 = random.randint(0,len(self.nets)-1)
            if(self.nets[k_1].score(Xc,yc)<self.nets[k_2].score(Xc,yc)):
                nets_new.append(self.nets[k_1])
            else:
                nets_new.append(self.nets[k_2])
        self.nets = copy.deepcopy(nets_new)
    
    def sort_nets(self,Xc,yc):
        # calculate score for each population of neural-net
        score_list = list(zip(self.nets, self.get_all_scores(Xc,yc)))

        # sort the network using its score
        score_list.sort(key=lambda x: x[1])

        # exclude score as it is not needed anymore
        score_list = [obj[0] for obj in score_list]
        self.nets = copy.deepcopy(score_list)
        if(self.best.accuracy(self.X,self.y)<self.nets[0].accuracy(self.X,self.y)):
            self.best = copy.deepcopy(self.nets[0])

    def evolve(self):
        start_time = time.time()
        for t in range(25):
            self.accuracy_train.append(self.best.accuracy(self.X,self.y))
            self.accuracy_test.append(self.best.accuracy(self.X_test,self.y_test))
            for i in range(100):
                j1=i*80
                j2=(1+i)*80
                Xc=self.X[j1:j2,:]
                yc=self.y[j1:j2,:]
                for k in range(25): 
                    self.crossover()
                    self.mutation()
                    self.selection(Xc,yc)
                    self.sort_nets(Xc,yc)
                print("Current iteration : {}, batch : {}".format(t+1,i+1))
                print("Time taken by far : %.1f seconds" % (time.time() - start_time))
                print("Current top member's network score: %.5f " % self.best.score(self.X,self.y))
                print("Current top member's network accuracy: %.2f%%\n" % self.best.accuracy(self.X,self.y))

In [4]:
df = pd.read_csv("../Data/data_8_8_yen.csv")

In [5]:
X = df.iloc[:8000, :8].values
y = df.iloc[:8000, 8:32].values
sc = StandardScaler()
X = sc.fit_transform(X)
X_test = df.iloc[8000:10000, :8].values
y_test = df.iloc[8000:10000, 8:32].values
sc = StandardScaler()
X_test = sc.fit_transform(X_test)

In [6]:
N_POPS = 100
NET_SIZE = [8,8,100,100,24]
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
nnga = NNGeneticAlgo(N_POPS, NET_SIZE, MUTATION_RATE, CROSSOVER_RATE, X, y, X_test, y_test)
nnga.evolve()

Current iteration : 1, batch : 1
Time taken by far : 171.4 seconds
Current top member's network score: 9.93760 
Current top member's network accuracy: 0.00%

Current iteration : 1, batch : 2
Time taken by far : 339.4 seconds
Current top member's network score: 9.93760 
Current top member's network accuracy: 0.00%

Current iteration : 1, batch : 3
Time taken by far : 508.6 seconds
Current top member's network score: 9.93760 
Current top member's network accuracy: 0.00%

Current iteration : 1, batch : 4
Time taken by far : 681.7 seconds
Current top member's network score: 9.93760 
Current top member's network accuracy: 0.00%

Current iteration : 1, batch : 5
Time taken by far : 854.2 seconds
Current top member's network score: 9.93760 
Current top member's network accuracy: 0.00%

Current iteration : 1, batch : 6
Time taken by far : 1037.9 seconds
Current top member's network score: 9.93760 
Current top member's network accuracy: 0.00%

Current iteration : 1, batch : 7
Time taken by far 

KeyboardInterrupt: 

In [ ]:
nnga.best.accuracy(X,y)

In [ ]:
nnga.best.accuracy(X_test,y_test)

In [ ]:
accuracy_train = np.array(nnga.accuracy_train)
accuracy_test = np.array(nnga.accuracy_test)

In [ ]:
plt.plot(accuracy_train)
plt.plot(accuracy_test )
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()